# 理解 Plonk（五）：多项式承诺

## 什么是多项式承诺

所谓承诺，是对消息「锁定」，得到一个锁定值。这个值被称为对象的「承诺」。

$$
c = commit(x)
$$

这个值和原对象存在两个关系，即 Hiding 与 Binding。

Hiding： $c$ 不暴露任何关于 $x$ 的信息；

Binding：难以找到一个 $x', x'\neq x$，使得 $c=commit(x')$。

最简单的承诺操作就是 Hash 运算。请注意这里的 Hash 运算需要具备密码学安全强度，比如 SHA256, Keccak 等。除了 Hash 算法之外，还有 Pedersen 承诺等。

顾名思义，多项式承诺可以理解为「多项式」的「承诺」。如果我们把一个多项式表达成如下的公式，

$$
f(X) = a_0 + a_1X + a_2X^2 + \cdots + a_nX^n
$$

那么我们可以用所有系数构成的向量来唯一标识多项式 $f(X)$。

$$
(a_0, a_1, a_2,\ldots, a_n)
$$

如何对一个多项式进行承诺？很容易能想到，我们可以把「系数向量」进行 Hash 运算，得到一个数值，就能建立与这个多项式之间唯一的绑定关系。

$$
C_1 = \textrm{SHA256}(a_0\parallel a_1 \parallel a_2 \parallel \cdots \parallel a_n)
$$

或者，我们也可以使用 Petersen 承诺，通过一组随机选择的基，来计算一个 ECC 点：

$$
C_2 = a_0 G_0 + a_1  G_1 + \cdots + a_n G_n
$$

如果在 Prover 承诺多项式之后，Verifier 可以根据这个承诺，对被锁定的多项式进行求值，并希望 Prover 可以证明求值的正确性。假设 $C=Commit(f(X))$，Verifier 可以向提供承诺的 Prover 询问多项式在 $X=\zeta$ 处的取值。Prover 除了回复一个计算结果之外（如 $f(\zeta) = y$） ，还能提供一个证明 $\pi$，证明 $C$ 所对应的多项式 $f(X)$ 在 $X=\zeta$ 处的取值 $y$ 的正确性。

多项式承诺的这个「携带证明的求值」特性非常有用，它可以被看成是一种轻量级的「可验证计算」。即 Verifier 需要把多项式 $f(X)$ 的运算代理给一个远程的机器（Prover），然后验证计算（计算量要小于直接计算 $f(X)$）结果 $y$ 的正确性；多项式承诺还能用来证明秘密数据（来自Prover）的性质，比如满足某个多项式，Prover 可以在不泄漏隐私的情况下向 Verifier 证明这个性质。

虽然这种可验证计算只是局限在多项式运算上，而非通用计算。但通用计算可以通过各种方式转换成多项式计算，从而依托多项式承诺来最终实现通用的可验证计算。

按上面 $C_2$ 的方式对多项式的系数进行 Pedersen 承诺，我们仍然可以利用 Bulletproof-IPA 协议来实现求值证明，进而实现另一种多项式承诺方案。此外，还有 KZG10 方案，FRI，Dark，Dory 等等其它方案。

## KZG10 构造

与 Pedersen 承诺中用的随机基向量相比，KZG10 多项式承诺需要用一组具有内部代数结构的基向量来代替。

$$
(G_0, G_1, G_2, \ldots, G_{d-1}, H_0, H_1) = (G, \chi G, \chi^2G, \ldots, \chi^{d-1}G, H, \chi H)
$$

请注意，这里的 $\chi$ 是一个可信第三方提供的随机数，也被称为 Trapdoor，需要在第三方完成 Setup 后被彻底删除。它既不能让 Verifier 知道，也不能让 Prover 知道。当 $\vec{G}$ 设置好之后， $\chi$ 被埋入了基向量中。这样一来，从外部看，这组基向量与随机基向量难以被区分。其中 $G\in\mathbb{G}_1$，而 $H\in\mathbb{G}_2$，并且存在双线性映射 $e\in \mathbb{G}_1\times\mathbb{G}_2\to \mathbb{G}_T$。

对于一个多项式 $f(X)$ 进行 KZG10 承诺，也是对其系数向量进行承诺：

$$
\begin{split}
C_{f(X)} &= a_0 G_0 + a_1  G_1 + \cdots + a_{n-1} G_{n-1} \\
 & = a_0  G + a_1 \chi G + \cdots + a_{n-1}\chi^{n-1} G\\
 & = f(\chi) G
\end{split}
$$

这样承诺 $C_{f(X)}$ 巧好等于 $f(\chi) G$。

对于双线性群，我们下面使用 Groth 发明的符号 $[1]_1\triangleq G$， $[1]_2\triangleq H$ 表示两个群上的生成元，这样 KZG10 的系统参数（也被称为 SRS, Structured Reference String）可以表示如下：

$$
\mathsf{srs}=([1]_1,[\chi]_1,[\chi^2]_1,[\chi^3]_1,\ldots,[\chi^{n-1}]_1,[1]_2,[\chi]_2)
$$

而 $C_{f(X)}=[f(\chi)]_1$。

下面使用上一节得到的多项式作为例子生成 commitment:
$$
f(X) = 40X^6 + 73X^5 + 32X^4 + 61X^2 + 28X + 69
$$


In [ ]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install py_ecc curve typing

In [152]:
import py_ecc.bn128 as b
from typing import NewType

G1Point = NewType("G1Point", tuple[b.FQ, b.FQ])
G2Point = NewType("G2Point", tuple[b.FQ2, b.FQ2])

class Setup():
    #   ([1]₁, [x]₁, ..., [x^{d-1}]₁)
    # = ( G,    xG,  ...,  x^{d-1}G ), where G is a generator of G_1
    powers_of_x: list[G1Point]
    # [x]₂ = xH, where H is a generator of G_2
    X2: G2Point

    # tau: a random number whatever you choose
    def __init__(self, powers: int, tau: int):
        print("Start to generate structured reference string")

        # Initialize powers_of_x with 0 values
        self.powers_of_x = [0] * powers
        # powers_of_x[0] =  b.G1 * tau**0 = b.G1
        # powers_of_x[1] =  b.G1 * tau**1 = powers_of_x[0] * tau
        # powers_of_x[2] =  b.G1 * tau**2 = powers_of_x[1] * tau
        # ...
        # powers_of_x[i] =  b.G1 * tau**i = powers_of_x[i - 1] * tau
        self.powers_of_x[0] = b.G1

        for i in range(powers):
            if i > 0:
                self.powers_of_x[i] = b.multiply(self.powers_of_x[i - 1], tau)

        print("Generated G1 side, X^1 point: {}".format(self.powers_of_x[1]))

        self.X2 = b.multiply(b.G2, tau)
        print("Generated G2 side, X^1 point: {}".format(self.X2))

        print("Finished to generate structured reference string")

    # Encodes the KZG commitment that evaluates to the given values in the group
    def commit(self, coeffs) -> G1Point:
        commitment = b.Z1
        for index, coeff in enumerate(coeffs):
            ec_point = b.multiply(self.powers_of_x[index], coeff % b.curve_order)
            commitment = b.add(commitment, ec_point)

        return commitment

In [153]:
# random number
tau = 74

# powers number should be larger than the degree of polynomial you want to commit
powers = 8

# do the setup
setup = Setup(powers, tau)

# combined polynomial combined_poly from last chapter:  40x^6 + 73x^5 + 32x^4 + 61x^2 + 28x + 69
f_coeffs = [69, 28, 61, 0, 32, 73, 40]

# commit
C_f = setup.commit(f_coeffs)
print("C_f: ", C_f)

assert C_f == (5944924129024846722741625252433644255755361576692942464620418422112209381657, 8343652485787411819127825636992247747330773247827507108642802455713543154102)

Start to generate structured reference string
Generated G1 side, X^1 point: (19000714569087058254079111722938672430276630300266312265196309930792761914189, 9925954159276340969458888695294901436812701424573926030176685839770908267539)
Generated G2 side, X^1 point: ((1143807547817528759872448485706976526436907032146971695798891422984531866726, 12471147282413329518352649295925560204177370150234424204094147917137349478087), (3714552804415258437881936892866262499095335352823341615390152505710682815003, 1775681321792385927298379567244671335181813357654154576426547499429277592831))
Finished to generate structured reference string
C_f:  (5944924129024846722741625252433644255755361576692942464620418422112209381657, 8343652485787411819127825636992247747330773247827507108642802455713543154102)


下面构造一个 $f(\zeta) = y$ 的 Open 证明。根据多项式余数定理，我们可以得到下面的等式：

$$
f(X) = q(X)\cdot (X-\zeta) + y
$$

这个等式可以解释为，任何一个多项式都可以除以另一个多项式，得到一个商多项式加上一个余数多项式。由于多项式在 $X=\zeta$ 处的取值为 $y$，那么我们可以确定：余数多项式一定为 $y$ ，因为等式右边的第一项在 $X=\zeta$ 处取值为零。所以，如果 $f(\zeta)=y$，我们可以断定： $g(X) = f(X)-y$ 在 $X=\zeta$ 处等零，所以 $\zeta$ 为 $g(X)$ 的根，于是 $g(X)$ 一定可以被 $(X-\zeta)$ 这个不可约多项式整除，即一定**存在**一个商多项式 $q(X)$，满足上述等式。

而 Prover 则可以提供 $q(X)$ 多项式的承诺，记为 $C_q$，作为 $f(\zeta)=y$ 的证明，Verifier 可以检查 $[q(\chi)]$ 是否满足整除性来验证证明。因为如果 $f(\zeta)\neq y$，那么 $g(X)$ 则无法被 $(X-\zeta)$ 整除，最终 Prover 提供的承诺无法通过整除性检查：

$$
(f(X)-y)\cdot 1 \overset{?}{=} q(X) \cdot (X-\zeta)
$$

承诺 $C_{f(X)}$ 是群 $\mathbb{G}_1$ 上的一个元素，通过承诺的加法同态映射关系，以及双线性映射关系 $e\in \mathbb{G}_1\times\mathbb{G}_2\to \mathbb{G}_T$，Verifier 可以在 $\mathbb{G}_T$ 上验证整除性关系：

$$
e(C_{f(X)} - y[1]_1, [1]_2) \overset{?}{=} e(C_{q(X)}, [\chi]_2 - \zeta [1]_2)
$$

有时为了减少 Verifier 在 $\mathbb{G}_2$ 上的昂贵操作，上面的验证等式可以变形为：

$$
f(X) + \zeta\cdot q(X) - y =  q(X)\cdot X
$$

$$
e(C_{f(X)} + \zeta\cdot C_{q(X)} -y\cdot[1]_1,\ [1]_2)\overset{?}{=} e(C_{q(X)},\  [\chi]_2)
$$


继续使用下面的多项式作为例子:
$$
f(X) = 40X^6 + 73X^5 + 32X^4 + 61X^2 + 28X + 69
$$

假设 $X=\zeta=1$，则 $f(\zeta)=y=303$

$q(X)=\frac{f(X)-y} {X-\zeta} = 40X^5 + 113X^4 + 145X^3 + 145X^2 + 206X + 234$

最终需要验证：

$$
e(C_{f(X)} + \zeta\cdot C_{q(X)} -y\cdot[1]_1,\ [1]_2)\overset{?}{=} e(C_{q(X)},\  [\chi]_2)
$$

下面我们使用代码来演示如何验证。

In [154]:
# random number
tau = 74

# powers number should be larger than the degree of polynomial you want to commit
powers = 8

# do the setup
setup = Setup(powers, tau)

# 40x^6 + 73x^5 + 32x^4 + 61x^2 + 28x + 69
f_coeffs = [69, 28, 61, 0, 32, 73, 40]

# random number
zeta = 1
# y = f(zeta), which is f(X) evaluate at X = zeta
y = 303

# q(X) = f(X)/(X-zeta) = 40X^5 + 113X^4 + 145X^3 + 145X^2 + 206X + 234
quot_coeffs = [234, 206, 145, 145, 113, 40]

C_f = setup.commit(f_coeffs)
C_q = setup.commit(quot_coeffs)
print("C_f: ", C_f)
print("C_q: ", C_q)

### do the linear combination for C_f + zeta * C_q - y * [1] ###

# zeta * C_q
C_linc_1 = b.multiply(C_q, zeta)
print("C_linc_1: ", C_linc_1)

# - y * [1]
assert -y % b.curve_order == 21888242871839275222246405745257275088548364400416034343698204186575808495314
assert b.G1 == setup.powers_of_x[0]
assert b.multiply(b.G2, tau) == setup.X2

C_linc_2 = b.multiply(b.G1, (-y) % b.curve_order)
assert C_linc_2 == (18788921900215882028576331803050066650912825348279745351731017781865893991129, 1073865593169846015410867644238358850025750640620148412555516903827118500108), "C_linc_2 not currect"
print("C_linc_2: ", C_linc_2)
assert b.multiply(C_f, 1) == C_f, "C_f should equal"

# final combination
C_linc = b.add(b.add(C_f, C_linc_1), C_linc_2)
print("C_linc: ", C_linc)
assert C_linc == (6301486514557274509058238130830016809489505352124366227551955002407651000262, 13048730005043404879416281954412709276285759583175910127109270903170794734865), "C_linc not correct"

assert b.pairing(b.G2, C_linc) == b.pairing(setup.X2, C_q), "Pairing not equal"

# no error, print success msg
print("Pairing check is successful!")

Start to generate structured reference string
Generated G1 side, X^1 point: (19000714569087058254079111722938672430276630300266312265196309930792761914189, 9925954159276340969458888695294901436812701424573926030176685839770908267539)
Generated G2 side, X^1 point: ((1143807547817528759872448485706976526436907032146971695798891422984531866726, 12471147282413329518352649295925560204177370150234424204094147917137349478087), (3714552804415258437881936892866262499095335352823341615390152505710682815003, 1775681321792385927298379567244671335181813357654154576426547499429277592831))
Finished to generate structured reference string
C_f:  (5944924129024846722741625252433644255755361576692942464620418422112209381657, 8343652485787411819127825636992247747330773247827507108642802455713543154102)
C_q:  (11740539305859663668512843267191890549789440068194701996652191809100675186622, 7923407303438355406747965479994761895007274674134214153265773933467813023745)
C_linc_1:  (117405393058596636685128432671

## 同点 Open 的证明聚合

在一个更大的安全协议中，假如同时使用多个多项式承诺，那么他们的 Open 操作可以合并在一起完成。即把多个多项式先合并成一个更大的多项式，然后仅通过 Open 一点，来完成对原始多项式的批量验证。

假设我们有多个多项式， $f_1(X)$， $f_2(X)$，Prover 要同时向 Verifier 证明 $f_1(\zeta)=y_1$ 和 $f_2(\zeta)=y_2$，那么有 

$$
\begin{array}{l}
f_1(X) = q_1(X)\cdot (X-\zeta) + y_1\\ 
f_2(X) = q_2(X) \cdot (X-\zeta) + y_2 \\
\end{array}
$$

通过一个随机数 $\nu$，Prover 可以把两个多项式 $f_1(X)$ 与 $f_2(X)$ 折叠在一起，得到一个临时的多项式 $g(X)$ ：

$$
g(X) = f_1(X) + \nu\cdot f_2(X)
$$

进而我们可以根据多项式余数定理，推导验证下面的等式：

$$
g(X) - (y_1 + \nu\cdot y_2) = (X-\zeta)\cdot (q_1(X) + \nu\cdot q_2(X))
$$

我们把等号右边的第二项看作为「商多项式」，记为 $q(X)$：

$$
q(X) = q_1(X) + \nu\cdot q_2(X)
$$

假如 $f_1(X)$ 在 $X=\zeta$ 处的求值证明为 $\pi_1$，而 $f_2(X)$ 在 $X=\zeta$ 处的求值证明为 $\pi_2$，那么根据群加法的同态性，Prover 可以得到商多项式  $q(X)$ 的承诺：

$$
[q(\chi)]_1 = \pi = \pi_1 + \nu\cdot\pi_2
$$

因此，只要 Verifier 发给 Prover 一个额外的随机数 $\nu$，双方就可以把两个（甚至多个）多项式承诺折叠成一个多项式承诺 $C_g$：

$$
C_g = C_1 + \nu\ast C_2
$$

并用这个折叠后的 $C_g$ 来验证多个多项式在一个点处的运算取值：

$$
y_g = y_1 + \nu\cdot y_2
$$

从而把多个求值证明相应地折叠成一个，Verifier 可以一次验证完毕：

$$
e(C-y\ast G_0, H_0) \overset{?}{=}e(\pi, H_1 - x\ast H_0)
$$

由于引入了随机数 $\nu$，因此多项式的合并不会影响承诺的绑定关系（Schwartz-Zippel 定理）。


### 协议：

公共输入： $C_{f_1}=[f_1(\chi)]_1$， $C_{f_2}=[f_2(\chi)]_1$， $\zeta$， $y_1$， $y_2$

私有输入： $f_1(X)$， $f_2(X)$

证明目标： $f_1(\zeta)=y_1$， $f_2(\zeta)=y_2$

第一轮：Verifier 提出挑战数 $\nu$

第二轮：Prover 计算 $q(X)=f_1(X)+\nu\cdot f_2(X)$，并发送  $\pi=[q(\chi)]_1$

第三轮：Verifier 计算 $C_g=C_{f_1} + \nu\cdot C_{f_2}$， $y_g = y_1 + \nu\cdot y_2$

$$
e(C_g - [y_g]_1, [1]_2)\overset{?}{=}e(\pi, [\chi-\zeta]_2)
$$



示例代码如下。

In [155]:
# random number
tau = 74

# powers number should be larger than the degree of polynomial you want to commit
powers = 8

# do the setup
setup = Setup(powers, tau)

# f1 = 4x^3 + 3x^2 + 2x + 1
f1_coeffs = [1, 2, 3, 4]
# f2 =  5x^3 + 4x^2 + 3x + 2
f2_coeffs = [2, 3, 4, 5]
C_f1 = setup.commit(f1_coeffs)
C_f2 = setup.commit(f2_coeffs)
print("C_f1: ", C_f1)
print("C_f2: ", C_f2)

# random number
zeta = 1
v = 2

# y = f(zeta), which is f(X) evaluate at X = zeta
y1 = 10
y2 = 14

# q(X) = f(X)/(X-zeta)
# (4x^3 + 3x^2 + 2x + 1 - y1)/(x-1) = 4x^2 + 7x + 9
quot1_coeffs = [9, 7, 4]
# (5x^3 + 4x^2 + 3x + 2 - y2)/(x-1) = 5x^2 + 9x + 12
quot2_coeffs = [12, 9, 5]

C_f1 = setup.commit(f1_coeffs)
C_f2 = setup.commit(f2_coeffs)
C_q1 = setup.commit(quot1_coeffs)
C_q2 = setup.commit(quot2_coeffs)
print("C_f1: ", C_f1)
print("C_f2: ", C_f2)
print("C_q1: ", C_q1)
print("C_q2: ", C_q2)


# compute combined values
# 1. compute g evaluation at zeta: y1 + v * y2
y = y1 + v * y2

# 2. compute g commitment
C_g = b.add(C_f1, b.multiply(C_f2, v))

# 3. compute g quotient commitment
C_q = b.add(C_q1, b.multiply(C_q2, v))

### do the linear combination for C_f + zeta * C_q - y * [1] ###

# zeta * C_q
C_linc_1 = b.multiply(C_q, zeta)
print("C_linc_1: ", C_linc_1)

# - y * [1]
assert b.G1 == setup.powers_of_x[0]
assert b.multiply(b.G2, tau) == setup.X2

C_linc_2 = b.multiply(b.G1, (-y) % b.curve_order)
print("C_linc_2: ", C_linc_2)
assert b.multiply(C_g, 1) == C_g, "C_g should equal"

# final combination
C_linc = b.add(b.add(C_g, C_linc_1), C_linc_2)
print("C_linc: ", C_linc)

assert b.pairing(b.G2, C_linc) == b.pairing(setup.X2, C_q), "Pairing not equal"

# no error, print success msg
print("Pairing check succeeded!")

Start to generate structured reference string
Generated G1 side, X^1 point: (19000714569087058254079111722938672430276630300266312265196309930792761914189, 9925954159276340969458888695294901436812701424573926030176685839770908267539)
Generated G2 side, X^1 point: ((1143807547817528759872448485706976526436907032146971695798891422984531866726, 12471147282413329518352649295925560204177370150234424204094147917137349478087), (3714552804415258437881936892866262499095335352823341615390152505710682815003, 1775681321792385927298379567244671335181813357654154576426547499429277592831))
Finished to generate structured reference string
C_f1:  (7685988750246226475753190070063637560822251204171270047630421061033479072206, 13893374176680490875221001856726590279934915186856450782578524321976046071072)
C_f2:  (8918628793504317541728615566017787534400742408097261238453518447333692861628, 10546747226089461796959884525324308807972154689294550154301849823609358250796)
C_f1:  (7685988750246226475753190070063


## 多项式约束与线性化

假设  $[f(\chi)]_1, [g(\chi)]_1, [h(\chi)]_1$ 分别是 $f(X),g(X),h(X)$ 的 KZG10 承诺，如果 Verifier 要验证下面的多项式约束：

$$
f(X) + g(X) \overset{?}{=} h(X)
$$

那么  Verifier 只需要把前两者的承诺相加，然后判断是否等于 $[h(\chi)]_1$  即可

$$
[f(\chi)]_1 + [g(\chi)]_1 \overset{?}{=} [h(\chi)]_1
$$

如果 Verifier 需要验证的多项式关系涉及到乘法，比如：

$$
f(X) \cdot g(X) \overset{?}{=} h(X)
$$

最直接的方法是利用双线性群的特性，在 $\mathbb{G}_T$ 上检查乘法关系，即验证下面的等式：

$$
e([f(\chi)]_1, [g(\chi)]_2) \overset{?}{=} e([h(\chi)]_1, [1]_2)
$$

但是如果 Verifier 只有 $g(X)$ 在 $\mathbb{G}_1$ 上的承诺 $[g(\chi)]_1$，而非是在 $\mathbb{G}_2$ 上的承诺 $[g(\chi)]_2$，那么Verifer 就无法利用双线性配对操作来完成乘法检验。

另一个直接的方案是把三个多项式在同一个挑战点 $X=\zeta$ 上打开，然后验证打开值之间的关系是否满足乘法约束：

$$
f(\zeta)\cdot g(\zeta)\overset{?}{=} h(\zeta)
$$

同时 Prover 还要提供三个多项式求值的证明 $(\pi_{f(\zeta)},\pi_{g(\zeta)},\pi_{h(\zeta)})$  供 Verifier 验证。 

这个方案的优势在于多项式的约束关系可以更加复杂和灵活，比如验证下面的稍微复杂些的多项式约束：

$$
f_1(X)f_2(X) + h_1(X)h_2(X)h_3(X) + g(X) = 0
$$

假设 Verifier 已拥有这些多项式的 KZG10 承诺， $[f_1(\chi)]_1$， $[f_2(\chi)]_1$， $[h_1(\chi)]_1$， $[h_2(\chi)]_1$， $[h_3(\chi)]_1$， $[g(\chi)]_1$。最直接粗暴的方案是让 Prover 在挑战点 $X=\zeta$ 处打开这 6 个承诺，发送 6 个 Open 值和对应的求值证明：

$$
(f_1(\zeta),\pi_{f_1}),(f_2(\zeta),\pi_{f_2}),(h_1(\zeta),\pi_{h_1}),(h_2(\zeta),\pi_{h_2}),(h_3(\zeta),\pi_{h_3}),(g(\zeta),\pi_{g})
$$

Verifier 验证 $6$ 个求值证明，并且验证多项式约束：

$$
f_1(\zeta)f_2(\zeta) + h_1(\zeta)h_2(\zeta)h_3(\zeta) + g(\zeta) \overset{?}{=} 0
$$

我们可以进一步优化，比如考虑对于 $f(X) \cdot g(X) = h(X)$ 这样一个简单的多项式约束，Prover 可以减少 Open 的数量。比如 Prover 先 Open $\bar{f} = f(\zeta)$，发送求值证明 $\pi_{f(\zeta)}$ 然后引入一个辅助多项式 $L(X)= \bar{f}\cdot g(X)-h(X)$，再 Open  $L(X)$ 在 $X=\zeta$ 处的取值。

显然对于一个诚实的 Prover， $L(\zeta)$ 求值应该等于零。对于 Verifier，它在收到 $\bar{f}$ 之后，就可以利用承诺的加法同态性，直接构造 $L(X)$ 的承诺：

$$
[L(\chi)]_1 = \bar{f}\cdot [g(\chi)]_1 - [h(\chi)]_1
$$

这样一来，Verifier 就不需要单独让 Prover 发送 $L(X)$ 的 Opening，也不需要发送新多项式 $L(X)$ 的承诺。Verifier 然后就可以验证 $f(X) \cdot g(X) = h(X)$ 这个多项式约束关系：

$$
e([L(\chi)]_1, [1]_2)\overset{?}{=} e(\pi_{L(\zeta)}, [\chi-\zeta]_2)
$$

这个优化过后的方案，Prover 只需要 Open 两次。第一个 Opening 为 $\bar{f}$，第二个 Opening 为 $0$。而后者是个常数，不需要发送给 Verifier。Prover 只需要发送两个求值证明，不过我们仍然可以用上一节提供的聚合证明的方法，通过一个挑战数 $\nu$，Prover 可以聚合两个多项式承诺，然后仅需要发送一个求值证明。



我们下面尝试优化下 $6$ 个多项式的约束关系的协议： $f_1(X)f_2(X) + h_1(X)h_2(X)h_3(X) + g(X) = 0$。

### 协议：

公共输入： $C_{f_1}=[f_1(\chi)]_1$， $C_{f_2}=[f_2(\chi)]_1$， $C_{h_1}=[h_1(\chi)]_1$， $C_{h_2}=[h_2(\chi)]_1$， $C_{h_3}=[h_3(\chi)]_1$， $C_{g}=[g(\chi)]_1$，

私有输入： $f_1(X)$， $f_2(X)$， $h_1(X)$， $h_2(X)$， $h_3(X)$， $g(X)$

证明目标： $f_1(X)f_2(X) + h_1(X)h_2(X)h_3(X) + g(X) = 0$

第一轮：Verifier 发送 $X=\zeta$

第二轮：Prover 计算并发送三个Opening， $\bar{f_1}=f_1(\zeta)$， $\bar{h}_1=h_1(\zeta)$， $\bar{h}_2=h_2(\zeta)$，

第三轮：Verifier 发送 $\nu$ 随机数

第四轮：Prover 计算 $L(X)$ ，利用 $\nu$ 折叠 $(L(X), f_1(X),h_1(X),h_2(X))$ 这四个承诺，并计算商多项式 $q(X)$，发送其承诺 $[q(\chi)]_1$ 作为折叠后的多项式在 $X=\zeta$ 处的求值证明

$$
L(X)=\bar{f}_1\cdot f_2(X) + \bar{h}_1\bar{h}_2\cdot h_3(X) + g(X)
$$

$$
q(X)=\frac{1}{X-\zeta}\Big(L(X) + \nu\cdot (f_1(X)-\bar{f}_1)+\nu^2\cdot (h_1(X)-\bar{h}_1)+\nu^3\cdot (h_2(X)-\bar{h}_2)\Big)
$$

第五轮：Verifier 计算辅助多项式 $L(X)$ 的承诺 $[L]_1$：

$$
[L]_1 = \bar{f}_1\cdot[f_2(\chi)]_1 + \bar{h}_1\bar{h}_2\cdot[h_3(\chi)]_1 + [g(\chi)]_1
$$

计算折叠后的多项式的承诺： 

$$
[F]_1=[L]_1 + \nu \cdot  [f_1(\chi)]_1+\nu^2[h_1(\chi)]_1+\nu^3[h_2(\chi)]_1
$$

计算折叠后的多项式在 $X=\zeta$  处的求值： 

$$
E=\nu\cdot \bar{f}_1 + \nu^2\cdot\bar{h}_1+ \nu^3\cdot\bar{h}_2
$$

检查下面的验证等式：

$$
e([F]_1-[E]_1 + \zeta[q(\chi)]_1, [1]_2)\overset{?}{=}e([q(\chi)]_1, [\chi]_2)
$$

这个优化后的协议，Prover 仅需要发送三个 Opening，一个求值证明；相比原始方案的 6 个 Opening和 6 个求值证明，大大减小了通信量（即证明大小）。


示例代码如下。

In [159]:
# random number
tau = 74

# powers number should be larger than the degree of polynomial you want to commit
powers = 8

# do the setup
setup = Setup(powers, tau)

# f1(X) = X
f1_coeffs = [1]
# f2(X) = X + 1
f2_coeffs = [1, 1]
# h1(X) = 2X
h1_coeffs = [0, 2]
# h2(X) = X + 2
h2_coeffs = [2, 1]
# h3(X) = X + 3
h3_coeffs = [3, 1]
# g(X) = -f1(X) * f2(X) - h1(X) * h2(X) * h3(X) = -2X^3 - 11X^2 - 13X
g_coeffs = [0, -13, -11, -2]

# random number
zeta = 3
f1 = 3
h1 = 6
h2 = 5

# L(x) = 3 * f2(X) + h1 * h2 * h3(X) + g(X) = -2X^3 - 11X^2 + 20X + 93
L_coeffs = [93, 20, -11, -2]

# random number
v = 2

# E(X) = v * f1(X) + v^2 * h1(X) + v^3 * h2(X) = 18X - 54
E_coeffs = [-54, 18]

# F(X) = L(x) + E(X) = -2X^3 - 11X^2 + 38X + 39
F_coeffs = [39, 38, -11, -2]

# F(zeta) == E(zeta) due to L(zeta) is always 0 for whatever value of zeta
# q(X) = (F(X) - E(zeta)) / (X - zeta) = −2X^2 − 17X − 13
q_coeffs = [-13, -17, -2]

E_at_zeta = 0

C_F = setup.commit(F_coeffs)
C_q = setup.commit(q_coeffs)

C_linc = b.add(C_F, b.multiply(C_q, zeta))

assert b.pairing(b.G2, C_linc) == b.pairing(setup.X2, C_q), "Pairing check failed"

print("Pairing check succeeded!")

Start to generate structured reference string
Generated G1 side, X^1 point: (19000714569087058254079111722938672430276630300266312265196309930792761914189, 9925954159276340969458888695294901436812701424573926030176685839770908267539)
Generated G2 side, X^1 point: ((1143807547817528759872448485706976526436907032146971695798891422984531866726, 12471147282413329518352649295925560204177370150234424204094147917137349478087), (3714552804415258437881936892866262499095335352823341615390152505710682815003, 1775681321792385927298379567244671335181813357654154576426547499429277592831))
Finished to generate structured reference string
Pairing check succeeded!
